In [1]:
from tensorflow.python.keras.utils.data_utils import get_file
import numpy as np
import os
import cv2
import tensorflow as tf
import time

In [2]:
# Set seed to get reproducable results

np.random.seed(123)

In [3]:
class Detector:
    def __init__(self):
        pass
    
    def readClasses(self, classesFilePath):
        with open(classesFilePath, 'r') as f:
            self.classesList = f.read().splitlines()
            
            # Colours List
            self.colorList = np.random.uniform(low=0, high=255, size=(len(self.classesList), 3))
            
            print(len(self.classesList), len(self.colorList))
            
    def downloadModel(self, modelURL):
        fileName = os.path.basename(modelURL)
        self.modelName = fileName[:fileName.index('.')]
        
        # Download and store it from the model zoo
        self.cacheDir = './pretrained_models'
        os.makedirs(self.cacheDir, exist_ok = True)
        get_file(fname=fileName, 
                origin=modelURL,
                cache_dir=self.cacheDir,
                cache_subdir='checkpoints',
                extract=True)
        
    def loadModel(self):
        print("Loading Model " + self.modelName)
        tf.keras.backend.clear_session() # Release memory from older models
        self.model = tf.saved_model.load(os.path.join(self.cacheDir, 'checkpoints', self.modelName, "saved_model"))

        print("Model " + self.modelName + " loaded successfully...")

    def createBoundingBox(self, image, threshold = 0.5):
        inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2BGR)
        inputTensor = tf.convert_to_tensor(inputTensor, dtype=tf.uint8) # Investigate tf.tensor objects
        inputTensor = inputTensor[tf.newaxis, ...] # What is happening here?

        detections = self.model(inputTensor) # Get predictions

        bboxs = detections['detection_boxes'][0].numpy()
        classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()

        imH, imW, imC = image.shape
        bboxIdx = tf.image.non_max_suppression(bboxs, classScores, max_output_size=50, iou_threshold=0.5, score_threshold=0.5)

        print(bboxIdx)
        
        
        if len(bboxIdx) != 0:
            for i in bboxIdx:
                bbox = tuple(bboxs[i].tolist())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]

                classLabelText = self.classesList[classIndex]
                classColor = self.colorList[classIndex]

                displayText = f'{classLabelText}: {classConfidence}%'

                ymin, xmin, ymax, xmax = bbox
                xmin, xmax, ymin, ymax = (xmin * imW, xmax*imW, ymin*imH, ymax*imH)
                xmin, xmax, ymin, ymax =  int(xmin), int(xmax), int(ymin), int(ymax)
                
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color = classColor, thickness=1)
                cv2.putText(image, displayText, (xmin, ymin-10), cv2.FONT_HERSHEY_PLAIN, 1, classColor, thickness=2)
                
        return image
                
            
            
    def predictImage(self, imagePath, threshold):
        image = cv2.imread(imagePath)
        bboxImage = self.createBoundingBox(image, threshold)

        cv2.imwrite(self.modelName + ".jpg", bboxImage)
        cv2.imshow("Result", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def predictVideo(self, videoPath, threshold = 0.5):
        cap = cv2.VideoCapture(videoPath)
        
        if(cap.isOpened() == False):
            print("Error opening file...")
            
        (success, image) = cap.read()

        startTime = 0

        while success:
            currentTime = time.time()

            fps = 1/(currentTime-startTime)
            startTime = currentTime
            bboxImage = self.createBoundingBox(image, threshold)
            cv2.putText(bboxImage, 'FPS: '+ str(int(fps)), (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            cv2.imshow('Result', bboxImage)

            key = cv2.waitKey(1) & 0xFF
            if key ==ord('q'):
                break

            (success, image) = cap.read()
                
        cv2.destroyAllWindows()
            

In [14]:
classFile = 'coco.names'
detector = Detector()
detector.readClasses(classFile)

modelURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'

# modelURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz'

92 92


In [15]:
detector.downloadModel(modelURL)
detector.loadModel()

imagePath = 'test/3.jpg'
videoPath = 'test/video-2.mp4' #0 for webcam
# videoPath = 0
threshold = 0.5
# detector.predictImage(imagePath, threshold)
detector.predictVideo(videoPath, threshold)

Loading Model ssd_mobilenet_v2_320x320_coco17_tpu-8
Model ssd_mobilenet_v2_320x320_coco17_tpu-8 loaded successfully...
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 12], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10]

tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 

tf.Tensor([0 1 2 3], shape=(4,), dtype=int32)
tf.Tensor([0 1 2 4 5], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 4 5], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 5 6], shape=(6,), dtype=int32)
tf.Tensor([0 1 2 3], shape=(4,), dtype=int32)
tf.Tensor([0 1 2 3 5], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 5], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4 6 7], shape=(7,), dtype=int32)
tf.Tensor([ 0  1  2  4  5  6  7  9 10], shape=(9,), dtype=int32)
tf.Tensor([0 1 3 4 5 6 7 8], shape=(8,), dtype=int32)
tf.Tensor([0 1 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12], shape=(13,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12], shape=(13,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12], shape=(13,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 9], shape=(9,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 11 12], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8 10 11 13], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([ 0  1  2  3  5  6  7  8  9 10], shape=(10,), dtyp

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8], shape=(9,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10], shape=(11,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  7  8  9 10 11 12], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 12], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12], shape=(13,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13], shape=(14,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  9 10 11 12 14 15], shape=(14,), dtype

...

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13], shape=(14,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], shape=(15,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17], shape=(18,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18], shape=(18,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21], shape=(22,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18], shape=(18,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 21 22 23 24], shape=(23,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22], shape=(22,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18], shape=(18,), dtype=int32)
tf.Tensor([ 0  1  2  3  4

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22], shape=(22,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16], shape=(16,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 19 20 21], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 19 20 21 22], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17], shape=(18,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22], shape=(22,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], shape=(15,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], shape=(21,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18], shape=(19,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15], shape=(16,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], shape=(20,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15], shape=(16,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], shape=(15,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13], shape=(14,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 11 12 13 14], shape=(14,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15], shape=(16,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16], shape=(17,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], shape=(15,), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12